In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx
import ray, importlib
import read, propagation
importlib.reload(propagation)

ModuleNotFoundError: No module named 'graphs'

In [ ]:
#ray.init()

In [ ]:
datadir = '/Users/ian/Nextcloud'
# datadir = '/home/sarming'
#read.adjlist(f'{datadir}/anonymized_outer_graph_neos_20200311.adjlist',
#              save_as=f'{datadir}/outer_neos.npz')
sim = propagation.Simulation.from_files(f'{datadir}/outer_neos.npz', f'{datadir}/authors_tweets_features_neos.csv' )
features = sim.features
params = sim.params
stats = sim.stats

In [ ]:
stats.max_retweets.max()

In [ ]:
np.array_split(features[0:10],32)


In [ ]:
import itertools, operator
stats.groupby('author_feature')['sources'].apply(lambda ls: set(filter(pd.notna,itertools.chain(*ls))))
#agg(operator.concat)

In [ ]:
sim.sources[('01000','1')]

In [ ]:
type(stats.loc[('0000','0000'),'tweets'])

In [ ]:
stats.info()

In [ ]:
stats.loc[('0000','0000')]

In [ ]:

sim.search_parameters(1,0.5)

In [ ]:
sim.search_parameters(200,0.0001)

In [ ]:
features.to_csv('features_200_0.0001.csv')

In [ ]:
csv=pd.read_csv(f'{datadir}/authors_tweets_features_neos.csv' )
csv['author_feature']=list(zip(csv.))

In [ ]:
list(params)

In [ ]:
[(x) for x,y in params]

In [ ]:
p=ray.get(params)

In [ ]:
 def helper(feature):
        sim.search_parameters(100,0.00001)
    sample_calls = [(A, coef, random_vector(n)) for _ in range(num_samples)]
s = sum(pool.starmap(chebyshev_sample, sample_calls))

In [ ]:
list(sim.simulate(sim.sample_feature(10)))

In [ ]:
sim.sample_source('0000',size=None)

In [ ]:
tweets.dropna().groupby('author_feature')['source'].unique()

In [ ]:
sim.sample_source('0000',1)[0]
# features.loc[('0000','0011')]

In [ ]:
af = features.groupby('author_feature')['tweets'].sum()
af /= af.sum()
np.random.choice(af.index,size=1,p=af)


In [ ]:
with pd.option_context("display.max_rows", 1000): display(sim.features)

In [ ]:
sim.sample_source('0000',size=None)

In [ ]:
tweets.dropna().groupby('author_feature')['source'].unique()

In [ ]:
sim.sample_source('0000',1)[0]
# features.loc[('0000','0011')]

In [ ]:
af = features.groupby('author_feature')['tweets'].sum()
af /= af.sum()
np.random.choice(af.index,size=1,p=af)


In [ ]:
with pd.option_context("display.max_rows", 1000): display(sim.features)